In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras import metrics

In [5]:
x_train_8 = np.load('207_data/saved_data/x_train_balanced_8_year.npy')
y_train_8 = np.load('207_data/saved_data/y_train_balanced_8_year.npy')

x_train_8 = np.reshape(x_train_8/255, (331240000, 3))
y_train_8 = y_train_8.flatten()

In [6]:
x_train_4 = np.load('207_data/saved_data/x_train_3500_balanced.npy')
y_train_4 = np.load('207_data/saved_data/y_train_3500_balanced.npy')

x_train_4 = np.reshape(x_train_4/255, (331240000, 3))
y_train_4 = y_train_4.flatten()

In [7]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)

base_model = tf.keras.Sequential()

    # Input layer
base_model.add(tf.keras.layers.Dense(
    units=3,                      # Number of neurons in the hidden layer
    input_shape=(3,),             # Input dimension
    activation='relu'             # Activation function (e.g., 'relu')
))
base_model.add(tf.keras.layers.Dense(
    units = 5,
    activation = 'relu'
))
base_model.add(tf.keras.layers.Dense(
    units = 5,
    activation = 'relu'
))

    # Output layer
base_model.add(tf.keras.layers.Dense(
    units=1,                      # Output dimension (1 for regression)
    activation='linear'           # Linear activation for regression
))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

base_model.compile(loss='mse', optimizer=optimizer)
base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 68
Trainable params: 68
Non-trainable params: 0
_________________________________________________________________


In [8]:
%%time

history = base_model.fit(
  x = x_train_4,
  y = y_train_4,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 1000,
  epochs = 5
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
298116/298116 [==============================] - 339s 1ms/step - loss: 509.7940 - val_loss: 102.9825
Epoch 2/5
298116/298116 [==============================] - 283s 950us/step - loss: 156.6893 - val_loss: 99.1763
Epoch 3/5
298116/298116 [==============================] - 279s 936us/step - loss: 154.5900 - val_loss: 98.7016
Epoch 4/5
298116/298116 [==============================] - 281s 942us/step - loss: 154.4574 - val_loss: 101.2368
Epoch 5/5
298116/298116 [==============================] - 266s 893us/step - loss: 154.6203 - val_loss: 98.3555


,loss,val_loss
0,292.549927,102.982491
1,156.088333,99.176292
2,154.686432,98.701591
3,154.777863,101.236839
4,154.601028,98.355545


Wall time: 24min 27s


In [9]:
tf.random.set_seed(0)
transfer_mod = tf.keras.Sequential()

base_model.trainable = False 

transfer_mod.add(base_model.layers[0])
transfer_mod.add(base_model.layers[1])


transfer_mod.add(tf.keras.layers.Dense(
    units = 3,
    activation = 'relu',
    name = 'dense_2'
))

transfer_mod.add(tf.keras.layers.Dense(
    units=1,                      # Output dimension (1 for regression)
    activation='linear',
    name = 'output'# Linear activation for regression
))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

transfer_mod.compile(loss='mse', optimizer=optimizer)
transfer_mod.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 18        
_________________________________________________________________
output (Dense)               (None, 1)                 4         
Total params: 54
Trainable params: 22
Non-trainable params: 32
_________________________________________________________________


In [12]:
%%time
history = transfer_mod.fit(
  x = x_train_8,
  y = y_train_8,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 1000,
  epochs=5
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
298116/298116 [==============================] - 301s 921us/step - loss: 257.2571 - val_loss: 111.2136
Epoch 2/5
298116/298116 [==============================] - 269s 902us/step - loss: 158.8532 - val_loss: 110.8866
Epoch 3/5
298116/298116 [==============================] - 251s 842us/step - loss: 158.4450 - val_loss: 110.9096
Epoch 4/5
298116/298116 [==============================] - 246s 826us/step - loss: 158.4363 - val_loss: 110.7327
Epoch 5/5
298116/298116 [==============================] - 241s 807us/step - loss: 158.3339 - val_loss: 110.6851


,loss,val_loss
0,181.486832,111.213562
1,158.994934,110.886620
2,158.787582,110.909622
3,158.683670,110.732697
4,158.648727,110.685104


Wall time: 22min 12s


In [14]:
tf.random.set_seed(0)
transfer_mod = tf.keras.Sequential()

base_model.trainable = False 

transfer_mod.add(base_model.layers[0])
transfer_mod.add(base_model.layers[1])
transfer_mod.add(base_model.layers[2])


transfer_mod.add(tf.keras.layers.Dense(
    units = 3,
    activation = 'relu'
))

transfer_mod.add(tf.keras.layers.Dense(
    units=1,                      # Output dimension (1 for regression)
    activation='linear',
    name = 'output'# Linear activation for regression
))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

transfer_mod.compile(loss='mse', optimizer=optimizer)
transfer_mod.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 18        
_________________________________________________________________
output (Dense)               (None, 1)                 4         
Total params: 84
Trainable params: 22
Non-trainable params: 62
_________________________________________________________________


In [15]:
%%time
history = transfer_mod.fit(
  x = x_train_8,
  y = y_train_8,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 1000,
  epochs=5
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
298116/298116 [==============================] - 316s 962us/step - loss: 180.7067 - val_loss: 109.3133
Epoch 2/5
298116/298116 [==============================] - 259s 868us/step - loss: 156.7113 - val_loss: 109.2208
Epoch 3/5
298116/298116 [==============================] - 269s 902us/step - loss: 156.6288 - val_loss: 109.2935
Epoch 4/5
298116/298116 [==============================] - 252s 846us/step - loss: 156.7241 - val_loss: 109.3014
Epoch 5/5
298116/298116 [==============================] - 250s 840us/step - loss: 156.6793 - val_loss: 109.3089


,loss,val_loss
0,160.604736,109.313255
1,156.988693,109.220818
2,156.989349,109.293533
3,156.988907,109.301384
4,156.989639,109.308868


Wall time: 22min 50s


In [16]:
%%time
history = transfer_mod.fit(
  x = x_train_8,
  y = y_train_8,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 1200,
  epochs=5
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
248430/248430 [==============================] - 377s 1ms/step - loss: 156.0879 - val_loss: 109.2996
Epoch 2/5
248430/248430 [==============================] - 316s 1ms/step - loss: 156.0899 - val_loss: 109.2311
Epoch 3/5
248430/248430 [==============================] - 324s 1ms/step - loss: 156.0904 - val_loss: 109.2897
Epoch 4/5
248430/248430 [==============================] - 306s 1ms/step - loss: 156.0918 - val_loss: 109.3056
Epoch 5/5
248430/248430 [==============================] - 309s 1ms/step - loss: 156.0903 - val_loss: 109.3048


,loss,val_loss
0,156.087860,109.299583
1,156.089905,109.231094
2,156.090439,109.289658
3,156.091827,109.305611
4,156.090332,109.304764


Wall time: 27min 35s


In [17]:
tf.random.set_seed(0)
transfer_mod = tf.keras.Sequential()

base_model.trainable = False 

transfer_mod.add(base_model.layers[0])
transfer_mod.add(base_model.layers[1])
transfer_mod.add(base_model.layers[2])


transfer_mod.add(tf.keras.layers.Dense(
    units = 3,
    activation = 'relu'
))

transfer_mod.add(tf.keras.layers.Dense(
    units=1,                      # Output dimension (1 for regression)
    activation='linear',
    name = 'output'# Linear activation for regression
))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

transfer_mod.compile(loss='mse', optimizer=optimizer)
transfer_mod.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 18        
_________________________________________________________________
output (Dense)               (None, 1)                 4         
Total params: 84
Trainable params: 22
Non-trainable params: 62
_________________________________________________________________


In [18]:
%%time
history = transfer_mod.fit(
  x = x_train_8,
  y = y_train_8,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 1000,
  epochs=5
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
298116/298116 [==============================] - 334s 1ms/step - loss: 325.7997 - val_loss: 109.2438
Epoch 2/5
298116/298116 [==============================] - 271s 907us/step - loss: 156.6982 - val_loss: 109.2631
Epoch 3/5
298116/298116 [==============================] - 274s 919us/step - loss: 156.6157 - val_loss: 109.2819
Epoch 4/5
298116/298116 [==============================] - 271s 910us/step - loss: 156.7117 - val_loss: 109.3056
Epoch 5/5
298116/298116 [==============================] - 240s 804us/step - loss: 156.6667 - val_loss: 109.2647


,loss,val_loss
0,196.783478,109.243828
1,156.975204,109.263092
2,156.976639,109.281921
3,156.976486,109.305618
4,156.975784,109.264717


Wall time: 23min 33s


In [19]:
tf.random.set_seed(0)
transfer_mod = tf.keras.Sequential()

base_model.trainable = False 

transfer_mod.add(base_model.layers[0])
transfer_mod.add(base_model.layers[1])
transfer_mod.add(base_model.layers[2])


transfer_mod.add(tf.keras.layers.Dense(
    units = 3,
    activation = 'relu'
))

transfer_mod.add(tf.keras.layers.Dense(
    units=1,                      # Output dimension (1 for regression)
    activation='linear',
    name = 'output'# Linear activation for regression
))

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

transfer_mod.compile(loss='mse', optimizer=optimizer)
transfer_mod.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 18        
_________________________________________________________________
output (Dense)               (None, 1)                 4         
Total params: 84
Trainable params: 22
Non-trainable params: 62
_________________________________________________________________


In [20]:
%%time
history = transfer_mod.fit(
  x = x_train_8,
  y = y_train_8,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 1000,
  epochs=5
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
298116/298116 [==============================] - 322s 975us/step - loss: 157.3354 - val_loss: 109.2433
Epoch 2/5
298116/298116 [==============================] - 272s 913us/step - loss: 156.5573 - val_loss: 109.0043
Epoch 3/5
298116/298116 [==============================] - 275s 921us/step - loss: 156.4324 - val_loss: 109.2412
Epoch 4/5
298116/298116 [==============================] - 268s 898us/step - loss: 156.5097 - val_loss: 109.0609
Epoch 5/5
298116/298116 [==============================] - 243s 815us/step - loss: 156.4629 - val_loss: 109.2753


,loss,val_loss
0,157.036133,109.243332
1,156.815231,109.004288
2,156.787064,109.241234
3,156.774170,109.060936
4,156.771805,109.275284


Wall time: 23min 24s


In [25]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)
transfer_mod = tf.keras.Sequential()

base_model.trainable = False 

transfer_mod.add(base_model.layers[0])
transfer_mod.add(base_model.layers[1])
transfer_mod.add(base_model.layers[2])


transfer_mod.add(tf.keras.layers.Dense(
    units = 2,
    activation = 'relu',
    name = 'dense_3'
))

transfer_mod.add(tf.keras.layers.Dense(
    units=1,                      # Output dimension (1 for regression)
    activation='linear',
    name = 'output'# Linear activation for regression
))

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

transfer_mod.compile(loss='mse', optimizer=optimizer)
transfer_mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 12        
_________________________________________________________________
output (Dense)               (None, 1)                 3         
Total params: 77
Trainable params: 15
Non-trainable params: 62
_________________________________________________________________


In [27]:
%%time
tf.random.set_seed(1000)
history = transfer_mod.fit(
  x = x_train_8,
  y = y_train_8,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 2000,
  epochs=5
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
149058/149058 [==============================] - 285s 2ms/step - loss: 156.5711 - val_loss: 109.2373
Epoch 2/5
149058/149058 [==============================] - 223s 1ms/step - loss: 156.5734 - val_loss: 109.2837
Epoch 3/5
149058/149058 [==============================] - 230s 2ms/step - loss: 156.5719 - val_loss: 109.3138
Epoch 4/5
149058/149058 [==============================] - 237s 2ms/step - loss: 156.5706 - val_loss: 109.2583
Epoch 5/5
149058/149058 [==============================] - 212s 1ms/step - loss: 156.5699 - val_loss: 109.2399


,loss,val_loss
0,156.571075,109.237335
1,156.573395,109.283691
2,156.571884,109.313782
3,156.570587,109.258278
4,156.569901,109.239914


Wall time: 20min 14s


In [23]:
%%time
tf.random.set_seed(0)
history = transfer_mod.fit(
  x = x_train_8,
  y = y_train_8,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 500,
  epochs=5
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
596232/596232 [==============================] - 733s 1ms/step - loss: 157.0862 - val_loss: 109.3273
Epoch 2/5
394346/596232 [==================>...........] - ETA: 3:37 - loss: 157.4346

KeyboardInterrupt: 

In [31]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)
transfer_mod = tf.keras.Sequential()

base_model.trainable = False 

transfer_mod.add(base_model.layers[0])
transfer_mod.add(base_model.layers[1])
transfer_mod.add(base_model.layers[2])


transfer_mod.add(tf.keras.layers.Dense(
    units = 100,
    activation = 'relu',
    name = 'dense_3'
))

# transfer_mod.add(tf.keras.layers.Dense(
#     units = 10,
#     activation = 'relu',
#     name = 'dense_3'
# ))

transfer_mod.add(tf.keras.layers.Dense(
    units=1,                      # Output dimension (1 for regression)
    activation='linear',
    name = 'output'# Linear activation for regression
))

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

transfer_mod.compile(loss='mse', optimizer=optimizer)
transfer_mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 100)               600       
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 763
Trainable params: 701
Non-trainable params: 62
_________________________________________________________________


In [32]:
%%time
tf.random.set_seed(0)
history = transfer_mod.fit(
  x = x_train_8,
  y = y_train_8,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 1000,
  epochs=5
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
298116/298116 [==============================] - 427s 1ms/step - loss: 156.9597 - val_loss: 107.8222
Epoch 2/5
298116/298116 [==============================] - 374s 1ms/step - loss: 155.1789 - val_loss: 107.2361
Epoch 3/5
298116/298116 [==============================] - 382s 1ms/step - loss: 154.8432 - val_loss: 107.2602
Epoch 4/5
298116/298116 [==============================] - 376s 1ms/step - loss: 154.7064 - val_loss: 107.0045
Epoch 5/5
298116/298116 [==============================] - 366s 1ms/step - loss: 154.4771 - val_loss: 107.7330


,loss,val_loss
0,156.281570,107.822220
1,155.371948,107.236130
2,155.131866,107.260193
3,154.914154,107.004486
4,154.746170,107.732971


Wall time: 32min 32s


In [35]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)
transfer_mod = tf.keras.Sequential()

base_model.trainable = False 

transfer_mod.add(base_model.layers[0])
transfer_mod.add(base_model.layers[1])
transfer_mod.add(base_model.layers[2])


transfer_mod.add(tf.keras.layers.Dense(
    units = 100,
    activation = 'relu',
    name = 'dense_3'
))

# transfer_mod.add(tf.keras.layers.Dense(
#     units = 10,
#     activation = 'relu',
#     name = 'dense_3'
# ))

transfer_mod.add(tf.keras.layers.Dense(
    units=1,                      # Output dimension (1 for regression)
    activation='linear',
    name = 'output'# Linear activation for regression
))

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

transfer_mod.compile(loss='mse', optimizer=optimizer)
transfer_mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 100)               600       
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 763
Trainable params: 701
Non-trainable params: 62
_________________________________________________________________


In [36]:
%%time
tf.random.set_seed(0)
history = transfer_mod.fit(
  x = x_train_8,
  y = y_train_8,
  validation_split=0.1,  # use 10% of the examples as a validation set
  batch_size = 1000,
  epochs=3
)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/3
298116/298116 [==============================] - 442s 1ms/step - loss: 156.9597 - val_loss: 107.8222
Epoch 2/3
298116/298116 [==============================] - 356s 1ms/step - loss: 155.1789 - val_loss: 107.2361
Epoch 3/3
298116/298116 [==============================] - 383s 1ms/step - loss: 154.8432 - val_loss: 107.2602


,loss,val_loss
0,156.281570,107.822220
1,155.371948,107.236130
2,155.131866,107.260193


Wall time: 20min 8s
